In [35]:
from haystack import Pipeline, component
from pymongo import MongoClient
from haystack.components.builders import PromptBuilder
from haystack_integrations.document_stores.mongodb_atlas import MongoDBAtlasDocumentStore
from haystack.utils import Secret
# from haystack.components.generators import Generator
from typing import List
from getpass import getpass
import os
import requests
import json

In [28]:
os.environ['MONGO_CONNECTION_STRING'] = getpass("Enter your MongoDB connection string: ")
os.environ['GROQ_API_KEY'] = getpass("Enter your GROQ API key: ")

In [29]:
class MongoDBAtlas : 
    def __init__(self, mongo_connection_string :str):
        self.client = MongoClient(mongo_connection_string)
        self.db = self.client.final_project
        self.categories_collection = self.db.categories
    
    def get_categories(self):
        return [doc['category'] for doc in self.categories_collection.find()]

In [30]:
@component
class GetCategories:
    def __init__(self):
        self.db = MongoDBAtlas(os.environ['MONGO_CONNECTION_STRING'])
    
    @component.output_types(categories=List[str])
    def run(self):
        categories = self.db.get_categories()
        return {"categories":categories}

In [41]:
from haystack import component
import requests

@component
class GroqLLM:
    def __init__(self, model_name="meta-llama/llama-4-maverick-17b-128e-instruct", api_key=None):
        self.api_key = api_key
        self.model_name = model_name

    @component.output_types(output=str)
    def run(self, prompt: str):
        url = "https://api.groq.com/openai/v1/chat/completions"
        headers = {
            "Authorization": f"Bearer {self.api_key}",
            "Content-Type": "application/json"
        }

        payload = {
            "model": self.model_name,
            "messages": [{"role": "user", "content": prompt}],
            "temperature": 0.7,
            "max_tokens": 300
        }

        response = requests.post(url, headers=headers, json=payload)
        try:
            data = response.json()
        except Exception:
            raise ValueError("Gagal parse JSON dari Groq API: ", response.text)

        # Debug untuk melihat isi JSON asli
        if "choices" not in data:
            raise ValueError(
                "Groq API tidak mengembalikan 'choices'.\n"
                f"Status Code: {response.status_code}\n"
                f"Response JSON:\n{json.dumps(data, indent=2)}"
            )

        # Jika OK, ambil isi respon
        result = data["choices"][0]["message"]["content"]
        return {"output": result}

In [ ]:
#create template 
TEMPLATE = """
You are a json generator that have job to generate json based on the input.
The return json should be in the format : 
```json
{
    "operator" : "AND",
    "conditions" : [
        {"field": "meta.category","operator" : "==", "value" : <category>}
    ]
}
The json key above can be comitted if the value is not provided in the input, so please make sure to only return the keys that are provided in the input.

For the material and category, you can only use the material and category that are provided below:
Categories: [ {% for category in categories %} {{ category }} {% if not loop.last %}, {% endif %} {% endfor %} ]

if the input does not contain any of the keys above, you should return an empty json object like this:
```json
{}
```
The example of the result are expected to be like this:

1. Input: "talk nonsense continue drama vote modi"
   output:
```json
{
    "conditions": [
        {"field": "meta.category", "operator": "==", "value": "Netral"}
    ]
}

2. Input: "things like demonetisation gst goods services taxthe upper castes would sort either view favourably say need give time castes like dalits muslims modi constituency2"
   output:
```json
{
    "conditions": [
        {"field": "meta.category", "operator": "==", "value": "Positive"}
    ]
}
3. Input: "vote party leadershipwho take fast firm action none narendra damodardas modi bjp party"
   output:
```json
{
    "conditions": [
        {"field": "meta.category", "operator": "==", "value": "Negative"}
    ]
}
4. Rules:
- DO NOT explain.
- DO NOT output Python code.
- DO NOT output steps.
- DO NOT output reasoning.
- Output ONLY valid JSON.

5. Input: {{input}}
output:

```

"""

In [43]:
groq_llm = GroqLLM(api_key=os.environ['GROQ_API_KEY'])

pipeline = Pipeline()
pipeline.add_component("categories", GetCategories())
pipeline.add_component(
    "prompt_builder",
    PromptBuilder(
        template=TEMPLATE,
        required_variables=["input", "categories"],
    )
)
pipeline.add_component("llm", groq_llm)
pipeline.connect("categories.categories", "prompt_builder.categories")
pipeline.connect("prompt_builder","llm.prompt")

🚅 Components
  - categories: GetCategories
  - prompt_builder: PromptBuilder
  - llm: GroqLLM
🛤️ Connections
  - categories.categories -> prompt_builder.categories (List[str])
  - prompt_builder.prompt -> llm.prompt (str)

In [44]:
groq_llm

llm
Inputs:
  - prompt: str
Outputs:
  - output: str

In [45]:
user_input = "Modi have vision to free india Corruption"
response = pipeline.run(
    {
        "prompt_builder":{
            "input": user_input
        }
    }
)

In [49]:
print(response['llm']['output'])

To solve this task, I will follow these steps:

1. Analyze the given input text.
2. Determine the category based on the input text.
3. Generate JSON according to the determined category.

First, let's analyze the examples given to understand how to determine the category:

1. Input: "talk nonsense continue drama vote modi" - Category: "Netral"
2. Input: "things like demonetisation gst goods services taxthe upper castes would sort either view favourably say need give time castes like dalits muslims modi constituency2" - Category: "Positive"
3. Input: "vote party leadershipwho take fast firm action none narendra damodardas modi bjp party" - Category: "Negative"

From the examples, it seems the category is determined based on the content of the input text, but the exact logic isn't clearly defined. For the purpose of this task, I'll assume a simple keyword-based approach where certain words are associated with categories. However, without a clear mapping or more detailed rules, I'll simpl